In [ ]:
# LLM HACKATHON 2025
# Data Cleaning Script for Alloy Datasets
# Author: Staradutt
import pandas as pd
import re

# -------------------------
# Helpers
# -------------------------
def normalize_formula(formula: str) -> str:
    if pd.isna(formula):
        return None
    formula = formula.replace(" ", "")
    parts = re.findall(r"[A-Z][a-z]?\d*\.?\d*", formula)
    parts = sorted(parts, key=lambda x: re.sub(r"\d.*", "", x))
    return "".join(parts)

def rename_cols_safe(df, mapping):
    return df.rename(columns={old: new for old, new in mapping.items() if old in df.columns})

def drop_element_columns(df, keep_cols):
    drop_cols = [c for c in df.columns if c not in keep_cols]
    return df.drop(columns=drop_cols)

# -------------------------
# Dataset 1 (MPEA)
# -------------------------
d1 = pd.read_csv("d1_mpea.csv")

map1 = {
    "IDENTIFIER: Reference ID": "id",
    "FORMULA": "formula",
    "PROPERTY: Microstructure": "microstructure",
    "PROPERTY: Processing method": "processing",
    "PROPERTY: BCC/FCC/other": "phase_label",
    "PROPERTY: grain size ($\\mu$m)": "grain_size",
    "PROPERTY: Exp. Density (g/cm$^3$)": "density_exp",
    "PROPERTY: Calculated Density (g/cm$^3$)": "density_calc",
    "PROPERTY: HV": "HV",
    "PROPERTY: YS (MPa)": "YS",
    "PROPERTY: UTS (MPa)": "UTS",
    "PROPERTY: Elongation (%)": "elongation",
    "PROPERTY: Exp. Young modulus (GPa)": "modulus_exp",
    "PROPERTY: Calculated Young modulus (GPa)": "modulus_calc",
    "REFERENCE: doi": "doi",
    "REFERENCE: year": "year",
    "REFERENCE: title": "title"
}
d1 = rename_cols_safe(d1, map1)
d1["composition_norm"] = d1["formula"].apply(normalize_formula)
d1.to_csv("dataset1_clean.csv", index=False)

# -------------------------
# Dataset 2 (ML Pred)
# -------------------------
d2 = pd.read_csv("d2_design.csv")

# Note the space in "Alloy "
map2 = {
    "Alloy ": "formula",  # <-- fix here
    "Alloy ID": "id",
    "Density_calc": "density_calc",
    "dHmix": "Hmix",
    "dSmix": "Smix",
    "dGmix": "Gmix",
    "Tm": "Tm",
    "Atom.Size.Diff": "delta",
    "Elect.Diff": "elec_diff",
    "VEC": "VEC",
    "Sythesis_Route": "processing",
    "Microstructure": "microstructure",
    "References": "doi"
}
d2 = rename_cols_safe(d2, map2)
d2["composition_norm"] = d2["formula"].apply(normalize_formula)

# Drop element fraction columns
keep_cols_d2 = list(map2.values()) + ["composition_norm", "id"]
d2 = drop_element_columns(d2, keep_cols_d2)
d2.to_csv("dataset2_clean.csv", index=False)

# -------------------------
# Dataset 3 (Achief)
# -------------------------
d3 = pd.read_csv("d3_param.csv")

map3 = {
    "Alloy": "formula",
    "delta": "delta",
    "Hmix": "Hmix",
    "Sid": "Smix",
    "VEC": "VEC",
    "BulkModulus (GPa)": "modulus_calc",
    "Tm (K)": "Tm",
    "Elec_nega": "elec_diff"
}
d3 = rename_cols_safe(d3, map3)
d3["composition_norm"] = d3["formula"].apply(normalize_formula)

keep_cols_d3 = list(map3.values()) + ["composition_norm","No","S_Phase","Phase","?Hmix (kJ/mol)","D_Tm (K)","D_Bulk (GPa)"]
d3 = drop_element_columns(d3, keep_cols_d3)
d3.to_csv("dataset3_clean.csv", index=False)

print("✅ Cleaned datasets saved separately: dataset1_clean.csv, dataset2_clean.csv, dataset3_clean.csv")


✅ Cleaned datasets saved separately: dataset1_clean.csv, dataset2_clean.csv, dataset3_clean.csv
